In [1]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast,AutoTokenizer
import torch
import evaluate
import pandas as pd

import wandb
from wandb.apis.public import Api

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device='cpu'

In [2]:

wandb.login()
api = Api()

run_id = '58advdgu'

runs = api.run(f"Kreol - NMT/runs/{run_id}")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: yush2398. Use `wandb login --relogin` to force relogin


In [3]:
for artifact_version in runs.logged_artifacts():
    # Replace with your own deletion logic.
    print(artifact_version.name.split(':')[-1])
    if artifact_version.name.split(':')[-1] == 'v0':
        artifact_version.download()

wandb: Downloading large artifact model-58advdgu:v0, 2352.49MB. 9 files... 


v0


wandb:   9 of 9 files downloaded.  
Done. 0:0:16.0


In [8]:
checkpoint = f"/mnt/disk/yrajcoomar/kreol-benchmark/pipelines/artifacts/model-{run_id}:v0"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

OSError: Incorrect path_or_model_id: '/mnt/disk/yrajcoomar/kreol-benchmark/pipelines/artifacts/model-5ol09fc9:v0'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
model = MBartForConditionalGeneration.from_pretrained(checkpoint)
model = model.to(device)

In [9]:
example_english_phrase = "Would you prefer money or shares in the company?"
# example_creole_phrase = 'Mo kosmar li koumanse aswar'

tokenizer.src_lang ='en_XX'
tokenizer.tgt_lang = 'cr_CR'

inputs = tokenizer(example_english_phrase, return_tensors="pt").to(device)

generated_tokens = model.generate(**inputs)

tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)


['Eski ou prefer larzan ou aksion dan konpayni?']

In [10]:
bleu = evaluate.load("bleu")
chrf = evaluate.load("chrf")

In [11]:
val = pd.read_json('/mnt/disk/yrajcoomar/kreol-benchmark/data/lang_data/en-cr/en-cr_dev.jsonl',lines=True)

tokenizer.src_lang='en_XX'
tokenizer.tgt_lang= 'cr_CR'

val_inputs = list(val['input'])
val_labels = list(val['target'])

In [12]:
input_tokens = tokenizer(val_inputs,max_length=128, truncation=True, padding="max_length",return_tensors='pt').to(device)
output_tokens = model.generate(**input_tokens)
output = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)

In [13]:
chrf.compute(predictions=output,references=val_labels) # unidir dictionary definitiosn + train

{'score': 45.04211625206542, 'char_order': 6, 'word_order': 0, 'beta': 2}

In [14]:
bleu.compute(predictions=output,references=val_labels) # unidir dictionary definitiosn + train

{'bleu': 0.2268658115491656,
 'precisions': [0.6021693491952415,
  0.31037899430270005,
  0.1810139952468973,
  0.10996466431095406],
 'brevity_penalty': 0.9185969191267758,
 'length_ratio': 0.9217372608041281,
 'translation_length': 8574,
 'reference_length': 9302}

In [2]:
str_v = '''
What vowel shows up the most often in the following paragraph: "Frankenstein is a frame story written in epistolary form. Set in the 18th century, it documents a fictional correspondence between Captain Robert Walton and his sister, Margaret Walton Saville. Robert Walton is a failed writer who sets out to explore the North Pole in hopes of expanding scientific knowledge. During the voyage, the crew spots a dog sled driven by a gigantic figure. A few hours later, the crew rescues a nearly frozen and emaciated man named Victor Frankenstein. Frankenstein has been in pursuit of the gigantic man observed by Walton's crew. Frankenstein starts to recover from his exertion; he sees in Walton the same obsession that has destroyed him and recounts a story of his life's miseries to Walton as a warning. The recounted story serves as the frame for Frankenstein's narrative." Respond with only the vowel and the number of occurrences, nothing else.
'''

In [3]:
str_v.count('e')

101